# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## Note: this is for use with 3d slices of data in Zimmer lab format 

This is for use with real WBFM data; for now I don't have annotations, and am just going to make sure the pipeline works for extracting volumes

## Create a new project

It is always good idea to keep the projects seperate if you want to use different networks to analze your data. You should use one project if you are tracking similar subjects/items even if in different environments. This function creates a new project with sub-directories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos (for lableing more data) to the project at any stage of the project. 

In [1]:
%env DLClight=True
import deeplabcut

env: DLClight=True


/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/users/

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
task='WormTest' # Enter the name of your experiment Task
experimenter='Charlie' # Enter the name of the experimenter
#video=['wb_immobilized/KU20170626-TKU721_nostim_ctrl_w2_part1.ome.tiff'] # Enter the paths of your videos OR FOLDER you want to grab frames from.
# video=['/groups/zimmer/Ulises/wbfm/wbfm_december2018/20181220/data/worm1/mCherry/ome']
video=['/users/charles.fieseler/test_worm1_data/mCherry/']

# path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=False, videotype='.tiff') 

# NOTE: The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
# path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-22/config.yaml'
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/config.yaml'
# path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-25/config.yaml'
#path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

In [3]:
#deeplabcut.create_new_project?

In [8]:
print(path_config_file)

/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/config.yaml


## Save individual volumes from ome.tiff movie

In [11]:
#
# Basically this is a custom extract_frames() function
#

from pathlib import Path
from deeplabcut.utils import auxiliaryfunctions
import tifffile
import platform
import os

In [10]:
def extract_volumes_from_many_files(path_config_file, which_vol=None):
    """
    Assumes a folder of many different volumes, and copies some in the folder given by 'out_folder'
    """
    
    if platform.system() is 'Windows':
        is_windows = True
    else:
        is_windows = False
    
    config_file = Path(path_config_file).resolve()
    cfg = auxiliaryfunctions.read_config(config_file)
    print("Config file read successfully.")
    
    video_fnames = [i for i in cfg['video_sets'].keys()]
    print(type(video_fnames))
    print("Found {} volumes.".format(len(video_fnames)))
    
    # Read basic metadata
    
    # Get volume indices to save
    if which_vol is None:
        which_vol = [0]
    
    for i_vol in which_vol:
        print("Reading volume {}/{}".format(i_vol, len(which_vol)))
        
        this_fname = video_fnames[i_vol]

        # Read and make output name
        this_volume = tifffile.imread(this_fname)
        output_name = 'img{}.tif'.format(i_vol)

        # Save in output folder
        fname = Path(video_fnames[0])
        output_path = os.path.join(Path(path_config_file).parents[0],'labeled-data',fname.stem)

        tifffile.imsave(os.path.join(str(output_path),output_name), this_volume)

        print('Saved volume to {}\\{}'.format(output_path, output_name))

In [11]:
        
def extract_volumes_from_MATLAB_output(path_config_file, which_vol=None):
    """
    Takes a video filename, which is a large ome-tiff file, and saves a volume in the folder given by 'out_folder'
    """
    
    if platform.system() is 'Windows':
        is_windows = True
    else:
        is_windows = False
    
    config_file = Path(path_config_file).resolve()
    cfg = auxiliaryfunctions.read_config(config_file)
    print("Config file read successfully.")
    
    video_fname = [i for i in cfg['video_sets'].keys()][0] # Assume one video for now (will be giant, ~3GB)
    print(video_fname)
    
    # Read basic metadata to get 'nz' and 'nt'
    with tifffile.TiffFile(video_fname) as vid:
        #print(tifffile.xml2dict(vid.ome_metadata))
        print(vid.ome_metadata)
        ome_metadata = vid.ome_metadata
        if isinstance(ome_metadata, str):
            # Appears to be a bug that just returns a string... can fix manually though
            ome_metadata = tifffile.xml2dict(ome_metadata)['OME']
        print(ome_metadata.keys())
        mdat = ome_metadata['Image']['Pixels']
        nz, nt = mdat['SizeZ'], mdat['SizeT']
    
    # Get volume indices to save
    if which_vol is None:
        which_vol = [0]
    
    for i_vol in which_vol:
        print("Read volume {}/{}".format(i_vol, len(which_vol)))
        
        # Convert scalar volume label to the sequential frames
        # Note: this may change for future input videos!
        vol_indices = list(range(i_vol*nz, i_vol*nz + nz))

        # Read and make output name
        this_volume = tifffile.imread(video_fname, key=vol_indices)
        output_name = 'img{}.tif'.format(i_vol)

        # Save in output folder
        fname = Path(video_fname)
        output_path = os.path.join(Path(path_config_file).parents[0],'labeled-data',fname.stem)

        tifffile.imsave(os.path.join(str(output_path),output_name), this_volume)

        print('Saved volume to {}\\{}'.format(output_path, output_name))
    

In [17]:
        
def extract_volumes_from_charlie_output(path_config_file, nz, which_vol=None):
    """
    Takes a video filename, which is a large ome-tiff file, and saves a volume in the folder given by 'out_folder'
    """
    
    config_file = Path(path_config_file).resolve()
    cfg = auxiliaryfunctions.read_config(config_file)
    print("Config file read successfully.")
    
    video_fname = [i for i in cfg['video_sets'].keys()][0] # Assume one video for now (will be giant, ~3GB)
    print(video_fname)
    
    # Get volume indices to save
    if which_vol is None:
        which_vol = [0]
    
    for i_vol in which_vol:
        print("Reading volume {}/{}".format(i_vol, len(which_vol)-1))
        
        # Convert scalar volume label to the sequential frames
        # Note: this may change for future input videos!
        vol_indices = list(range(i_vol*nz, i_vol*nz + nz))
        print("Converted volume indices: {} to {} (not including last frame)".format(i_vol*nz, i_vol*nz + nz))

        # Read and make output name
        this_volume = tifffile.imread(video_fname, key=vol_indices)
        output_name = 'img{}.tif'.format(i_vol)

        # Save in output folder
        fname = Path(video_fname)
        output_path = os.path.join(Path(path_config_file).parents[0],'labeled-data',fname.stem)

        tifffile.imsave(os.path.join(str(output_path),output_name), this_volume)

        print('Saved volume to {}'.format(os.path.join(output_path, output_name)))
    

In [20]:
extract_volumes_from_charlie_output(path_config_file, 39, list(range(5)))

Config file read successfully.
/users/charles.fieseler/test_worm1_data/mCherry/test_100frames.ome.tiff
Reading volume 0/4
Converted volume indices: 0 to 39 (not including last frame)
Saved volume to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img0.tif
Reading volume 1/4
Converted volume indices: 39 to 78 (not including last frame)
Saved volume to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img1.tif
Reading volume 2/4
Converted volume indices: 78 to 117 (not including last frame)
Saved volume to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img2.tif
Reading volume 3/4
Converted volume indices: 117 to 156 (not including last frame)
Saved volume to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img3

# Separately: Manually do BCPD to create the annotations!

For now, is completely separate

# Automatically update the config file

# TODO; use edit_config()

In [3]:
import csv
import h5py
import pandas as pd
import os

In [4]:
def bpcd_tracker2config_names(path_config_file):
    """
    Automatically updates the config file with the proper number of neurons, and deletes any other default bodyparts.
    Only affects the "bodyparts" field
    """
    
    # Get number of neurons from annotations
    home = os.path.dirname(path_config_file)
    # TODO: hardcoded folder
    annotations_fname = os.path.join(home,'labeled-data', 'test_100frames.ome','CollectedData_Charlie.csv')
    df = pd.read_csv(annotations_fname)
    num_neurons = int(df.shape[1] / 3)
    print("Adding body part annotations for {} neurons".format(num_neurons))
#     error()
    
    # Read in entire config file into a list
    config_rows = []
    with open(path_config_file) as config:
        c_reader = csv.reader(config)#, delimiter=' ')
        for row in c_reader:
            config_rows.append(row)
    
    ## Delete the current bodypart lines
    delete_these_rows = False
    config_rows_edit = config_rows.copy()
    for row in config_rows:
        if row == ['bodyparts:']:
            delete_these_rows = True # Start deleting next row
        elif row == ['start: 0']:
            delete_these_rows = False # Do not delete this row, or others
            break
        elif delete_these_rows == True:
            # Don't delete either of the two above, but only in between those rows
            config_rows_edit.remove(row)
    
    ## Add in the named neuron lines
    # Using "list slicing" https://www.geeksforgeeks.org/python-insert-list-in-another-list/
    new_names = [['- neuron{}'.format(i)] for i in range(num_neurons)]
    insert_index = config_rows_edit.index(['start: 0'])
    config_rows_edit[insert_index:insert_index] = new_names
    
    ## Write the file again
    if True:
        with open(path_config_file, 'w', newline='') as config:
            c_writer = csv.writer(config)
            for row in config_rows_edit:
                c_writer.writerow(row)
    
    print("Finished! Check the config.yaml file to make sure the bodyparts are properly written")
    

In [5]:
def wb_tracker2config_names(path_config_file):
    """
    Automatically updates the config file with the proper number of neurons, and deletes any other default bodyparts.
    Only affects the "bodyparts" field
    """
    
    # Get number of neurons from annotations
    home = os.path.dirname(path_config_file)
    # TODO: hardcoded folder
    annotations_fname = os.path.join(home,'labeled-data', 'my-test-annotations','CollectedData_Charlie.csv')
#     with h5py.File(annotations_fname, 'r') as ann:
#         k = list(ann['df_with_missing']['table'].keys())
#         print(k)
#         num_neurons = int(len(ann[k[0]])/3)
    df = pd.read_csv(annotations_fname)
    num_neurons = int(df.shape[1] / 3)
    print("Adding body part annotations for {} neurons".format(num_neurons))
#     error()
    
    # Read in entire config file into a list
    config_rows = []
    with open(path_config_file) as config:
        c_reader = csv.reader(config)#, delimiter=' ')
        for row in c_reader:
            config_rows.append(row)
    
    ## Delete the current bodypart lines
    delete_these_rows = False
    config_rows_edit = config_rows.copy()
    for row in config_rows:
        if row == ['bodyparts:']:
            delete_these_rows = True # Start deleting next row
        elif row == ['start: 0']:
            delete_these_rows = False # Do not delete this row, or others
            break
        elif delete_these_rows == True:
            # Don't delete either of the two above, but only in between those rows
            config_rows_edit.remove(row)
    
    ## Add in the named neuron lines
    # Using "list slicing" https://www.geeksforgeeks.org/python-insert-list-in-another-list/
    new_names = [['- neuron{}'.format(i)] for i in range(num_neurons)]
    insert_index = config_rows_edit.index(['start: 0'])
    config_rows_edit[insert_index:insert_index] = new_names
    
    ## Write the file again
    if True:
        with open(path_config_file, 'w', newline='') as config:
            c_writer = csv.writer(config)
            for row in config_rows_edit:
                c_writer.writerow(row)
    #for row in config_rows_edit:
        #c_writer.writerow(row)
     #   print(row[:])
    
    

In [25]:
bpcd_tracker2config_names(path_config_file)

Adding body part annotations for 34 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written


## Check the labels

[OPTIONAL] Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [7]:
%env DLClight=True
import deeplabcut
# path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/config.yaml'


env: DLClight=True


In [21]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

  0%|          | 0/5 [00:00<?, ?it/s]

Creating images with labels by Charlie.
Debug; reading from files:  Index(['/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img0.tif',
       '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img1.tif',
       '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img2.tif',
       '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img3.tif',
       '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome/img4.tif'],
      dtype='object')
/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/labeled-data/test_100frames.ome_labeled  already exists!


100%|██████████| 5/5 [00:01<00:00,  4.15it/s]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use relauch the labeling GUI to move them around, save, and re-plot!

## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [1]:
# IF THE KERNEL IS RESET
%env DLClight=True
import deeplabcut
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/config.yaml'
# path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-22/config.yaml'
#path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

env: DLClight=True


/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/users/

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [10]:
deeplabcut.create_training_dataset(path_config_file)
#remember, there are several networks you can pick, the default is resnet-50!

Recognized z-slice training data; using custom function
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95, 1, (array([4, 3, 0, 1]), array([2])))]

In [202]:
deeplabcut.create_training_dataset?

## Start training:

This function trains the network for a specific shuffle of the training dataset. 

In [2]:
deeplabcut.train_network(path_config_file, displayiters=100, saveiters=1000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26],
                [27],
                [28],
                [29],
                [30],
                [31],
                [32],
                [33]],
 'all_joints_names': ['neuron0',
                      'neuron1',
                      'neuron2',
                      'neuron3',
                      'neuron4',
                      'neuron5',
                      'neuron6',
                      

Selecting single-animal trainer
Switching batchsize to 1, as default/tensorpack/deterministic loaders do not support batches >1. Use imgaug loader.
Starting with standard pose-dataset loader.
Initializing PoseNetSlices
Creating new class for use with z-slice data, PoseNetSlices
Loading ImageNet-pretrained resnet_50
Display_iters overwritten as 100
Save_iters overwritten as 1000
Training parameter:
{'using_z_slices': True, 'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/dlc-models/iteration-1/WormTestJun30-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 1.0, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 

InvalidArgumentError: Image width 320 and height 244 should be divisible by block_size: 3
	 [[node pose/locref_pred/map/while/SpaceToDepth (defined at /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/deeplabcut/pose_estimation_tensorflow/nnet/pose_net_slices.py:73) ]]
	 [[{{node GroupCrossDeviceControlEdges_0/absolute_difference/weighted_loss/num_present/broadcast_weights/assert_broadcastable/AssertGuard/Assert/data_7}}]]

Caused by op 'pose/locref_pred/map/while/SpaceToDepth', defined at:
  File "/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/users/charles.fieseler/.conda/envs/DLC-GPU-dev/lib/python3.6/site

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [5]:
# IF THE KERNEL IS RESET
%env DLClight=True
import deeplabcut
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/config.yaml'

#path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

env: DLClight=True


In [9]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26],
                [27],
                [28],
                [29],
                [30],
                [31],
                [32],
                [33]],
 'all_joints_names': ['neuron0',
                      'neuron1',
                      'neuron2',
                      'neuron3',
                      'neuron4',
                      'neuron5',
                      'neuron6',
                      

/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/evaluation-results/  already exists!
/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/evaluation-results/iteration-1/WormTestJun30-trainset95shuffle1  already exists!
Running  DLC_resnet50_WormTestJun30shuffle1_12000  with # of trainingiterations: 12000
Initializing ResNet


0it [00:00, ?it/s]

Analyzing data...


5it [00:00,  8.85it/s]
/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/deeplabcut/pose_estimation_tensorflow/evaluate.py:857: RuntimeWarning: Mean of empty slice
  RMSEpcutoff.iloc[testIndices].values.flatten()
/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/deeplabcut/pose_estimation_tensorflow/evaluate.py:860: RuntimeWarning: Mean of empty slice
  RMSEpcutoff.iloc[trainIndices].values.flatten()
  0%|          | 0/5 [00:00<?, ?it/s]

Done and results stored for snapshot:  snapshot-12000
Results for 12000  training iterations: 95 1 train error: 358.75 pixels. Test error: 379.51  pixels.
With pcutoff of 0.6  train error: nan pixels. Test error: nan pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


100%|██████████| 5/5 [00:01<00:00,  2.51it/s]

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [3]:
videofile_path = ['videos/video3.avi','videos/video4.avi'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

Using snapshot-11000 for model /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/dlc-models/iteration-0/WormTestJun12-trainset95shuffle1
Initializing ResNet
No video(s) were found. Please check your paths and/or 'video_type'.


'DLC_resnet50_WormTestJun12shuffle1_11000'

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [3]:
deeplabcut.extract_outlier_frames?

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [ ]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)